In [1]:
'''Main'''
import numpy as np
import pandas as pd
import os

'''Data Viz'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''Data Prep'''
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.compose import ColumnTransformer 
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import precision_recall_curve, average_precision_score,precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report 

'''Algos'''
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import autogluon.core as ag
from autogluon import TabularPrediction as task
#import xgboost as xgb
#import lightgbm as lgb

# Data Preparation

## Acquire Data

In [2]:
# We only get the train data set
current_path = os.getcwd()+"\\"
file = os.path.sep.join(["..\\data\\train.csv"])
data = pd.read_csv(current_path + file)
data=data.set_index("Id")

# signal is the target variable
df=data.drop(["BUTTER"], axis=1) # Droping BUTTER variable as is IRRELEVANT

In [3]:
msk = np.random.rand(len(df)) < 0.95

train = df[msk]

test = df[~msk]

# AutoGluon Inital

In [52]:
train_data = task.Dataset(train)
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)

In [60]:
label_column = 'signal'
print("Summary of class variable: \n", train_data[label_column].describe())

Summary of class variable: 
 count    210568.000000
mean          0.334001
std           0.471641
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: signal, dtype: float64


In [30]:
dir = current_path+"\\trained_models"
predictor = task.fit(train_data=train_data, label=label_column, output_directory=dir)

Beginning AutoGluon training ...
AutoGluon will save models to C:\Users\garciagr\OneDrive - HP Inc\DSUB\MachineLearning\Boson\2020.ml.kaggle\Marc\\trained_models\
AutoGluon Version:  0.0.14
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    39187.19 MB
	Train Data (Original)  Memory Usage: 0.06 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.


In [32]:
test_data = task.Dataset(test)
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(labels=[label_column],axis=1)  # delete label column to prove we're not cheating

In [39]:
predictor = task.load(dir)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.7050730063646574
Evaluations on test data:
{
    "accuracy": 0.7050730063646574,
    "accuracy_score": 0.7050730063646574,
    "balanced_accuracy_score": 0.5768322523659386,
    "matthews_corrcoef": 0.2510302944295476,
    "f1_score": 0.7050730063646574
}
Detailed (per-class) classification report:
{
    "0.0": {
        "precision": 0.7050793323717288,
        "recall": 0.9595485137408861,
        "f1-score": 0.812863760541632,
        "support": 14264
    },
    "1.0": {
        "precision": 0.7050102249488752,
        "recall": 0.194115990990991,
        "f1-score": 0.30441501103752766,
        "support": 7104
    },
    "accuracy": 0.7050730063646574,
    "macro avg": {
        "precision": 0.705044778660302,
        "recall": 0.5768322523659386,
        "f1-score": 0.5586393857895798,
        "support": 21368
    },
    "weighted avg": {
        "precision": 0.7050563569350033,
        "recall": 0.7050730063646574,
        "f1-score": 0.6438250

Predictions:   [0 1 1 ... 0 0 0]


In [35]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesClassifierEntr,0.739985,0.71,0.224589,0.105948,0.302738,0.224589,0.105948,0.302738,0,True,4
1,ExtraTreesClassifierGini,0.739798,0.71,0.226053,0.105911,0.345051,0.226053,0.105911,0.345051,0,True,3
2,LightGBMClassifierXT,0.739657,0.74,0.049918,0.003709,0.274369,0.049918,0.003709,0.274369,0,True,8
3,RandomForestClassifierGini,0.738628,0.70,0.114872,0.105667,0.337162,0.114872,0.105667,0.337162,0,True,1
4,RandomForestClassifierEntr,0.737973,0.69,0.123669,0.105312,0.327921,0.123669,0.105312,0.327921,0,True,2
5,CatboostClassifier,0.737364,0.72,0.008056,0.003978,0.915800,0.008056,0.003978,0.915800,0,True,9
6,NeuralNetClassifier,0.718925,0.72,0.524147,0.012079,5.260593,0.524147,0.012079,5.260593,0,True,10
7,LightGBMClassifierCustom,0.709940,0.73,0.015163,0.002543,0.354200,0.015163,0.002543,0.354200,0,True,11
8,weighted_ensemble_k0_l1,0.705073,0.77,0.024959,0.006476,0.883696,0.003812,0.000000,0.272763,1,True,12
9,LightGBMClassifier,0.704558,0.76,0.005984,0.003932,0.256732,0.005984,0.003932,0.256732,0,True,7


# AutoGluon Maxime accuracy

In [4]:
train_data = task.Dataset(train)
test_data = task.Dataset(test)
label_column="signal"
output_directory=current_path+ "\\trained_models"

In [6]:
time_limits = 28800 # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc' # specify your evaluation metric here
#presets="medium_quality_faster_train" # produces less accurate models but facilitates faster prototyping
presets='best_quality'
predictor = task.fit(train_data=train_data, label=label_column, time_limits=time_limits,
                     eval_metric=metric, presets=presets,output_directory=output_directory,
                    num_bagging_folds=6, num_bagging_sets=3)

Beginning AutoGluon training ... Time limit = 28800s
AutoGluon will save models to D:\Uni\2020.ml.kaggle\Marc\\trained_models\
AutoGluon Version:  0.0.14
Train Data Rows:    202150
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1.0, 0.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    29442.57 MB
	Train Data (Original)  Memory Usage: 24.26 MB (0.1% of available memory)
	Inferring data type of each feature ba

	11.56s	 = Validation runtime
Fitting model: ExtraTreesClassifierEntr_STACKER_l1 ... Training model for up to 11183.59s of the 11183.53s of remaining time.
	0.9182	 = Validation roc_auc score
	606.67s	 = Training runtime
	11.42s	 = Validation runtime
Fitting model: KNeighborsClassifierUnif_STACKER_l1 ... Training model for up to 10865.76s of the 10865.69s of remaining time.
	0.677	 = Validation roc_auc score
	47.1s	 = Training runtime
	7.98s	 = Validation runtime
Fitting model: KNeighborsClassifierDist_STACKER_l1 ... Training model for up to 10838.71s of the 10838.65s of remaining time.
	0.6782	 = Validation roc_auc score
	41.36s	 = Training runtime
	6.61s	 = Validation runtime
Fitting model: LightGBMClassifier_STACKER_l1 ... Training model for up to 10813.94s of the 10813.88s of remaining time.
	0.9196	 = Validation roc_auc score
	21.51s	 = Training runtime
	0.43s	 = Validation runtime
Fitting model: LightGBMClassifierXT_STACKER_l1 ... Training model for up to 10802.36s of the 10802.3

In [7]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetClassifier_STACKER_l0,0.926668,0.919258,2.501112,5.180921,8667.302806,2.501112,5.180921,8667.302806,0,True,10
1,weighted_ensemble_k0_l2,0.926296,0.919954,252.268183,86.256277,23779.056685,0.012963,0.061362,70.406935,2,True,24
2,LightGBMClassifier_STACKER_l1,0.926279,0.919718,168.390095,43.264239,10696.130525,0.501668,0.659074,32.645846,1,True,19
3,weighted_ensemble_k0_l1,0.926206,0.919478,6.172467,10.867985,9241.919919,0.026931,0.061893,72.975587,1,True,12
4,CatboostClassifier_STACKER_l1,0.926192,0.919755,168.117999,42.809623,10768.037492,0.229573,0.204458,104.552814,1,True,21
5,LightGBMClassifierXT_STACKER_l1,0.926154,0.919775,168.648414,43.622747,10705.825055,0.759988,1.017583,42.340376,1,True,20
6,LightGBMClassifierCustom_STACKER_l1,0.926087,0.919638,169.656273,44.853723,10766.261203,1.767847,2.248558,102.776525,1,True,23
7,NeuralNetClassifier_STACKER_l1,0.925999,0.919333,177.080240,58.123583,19205.332753,9.191814,15.518418,8541.848074,1,True,22
8,RandomForestClassifierEntr_STACKER_l1,0.924438,0.918428,202.806513,53.553821,13068.591482,34.918087,10.948657,2405.106803,1,True,14
9,ExtraTreesClassifierEntr_STACKER_l1,0.923738,0.918354,270.372780,59.314321,11566.359820,102.484354,16.709156,902.875142,1,True,16


## Add aditional ensembles

In [18]:
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True, silent=True)

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\utils\data\X_train.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\utils\data\y_train.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierEntr_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierGini_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierDist_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_STACKER_l

	0.915	 = Validation roc_auc score
	66.93s	 = Training runtime
	0.06s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Fitting model: weighted_ensemble_custom_pareto10_k0_l1 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto10_k0_l1\utils\model_template.pkl
Ensemble size: 57
Ensemble weights: 
[0.96491228 0.03508772 0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto10_k0_l1\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto10_k0_l1\model.pkl
	0.915	 = Validation roc_auc score
	73.39s	 = Training runtime
	0.06s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Fitting model: weighted_ensemble_custom_pareto11_k0_l2 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_e

	0.9153	 = Validation roc_auc score
	134.27s	 = Training runtime
	0.07s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Fitting model: weighted_ensemble_custom_k0_l2 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_k0_l2\utils\model_template.pkl
Ensemble size: 73
Ensemble weights: 
[0.23287671 0.15068493 0.1369863  0.17808219 0.06849315 0.12328767
 0.08219178 0.02739726 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_k0_l2\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_k0_l2\model.pkl
	0.9153	 = Validation roc_auc score
	139.98s	 = Training runtime
	0.06s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl


Alternative ensembles you can use for prediction: ['weighted_ensemble_custom_pareto1_k0_l1', 'weighted_ensemble_custom_pareto2_k0_l1', 'weighted_ensemble_custom_pareto3_k0_l1', 'weighted_ensemble_custom_pareto4_k0_l1', 'weighted_ensemble_custom_pareto5_k0_l1', 'weighted_ensemble_custom_pareto6_k0_l1', 'weighted_ensemble_custom_pareto7_k0_l1', 'weighted_ensemble_custom_pareto8_k0_l1', 'weighted_ensemble_custom_pareto9_k0_l1', 'weighted_ensemble_custom_pareto10_k0_l1', 'weighted_ensemble_custom_pareto11_k0_l2', 'weighted_ensemble_custom_pareto12_k0_l2', 'weighted_ensemble_custom_pareto13_k0_l2', 'weighted_ensemble_custom_pareto14_k0_l2', 'weighted_ensemble_custom_pareto15_k0_l2', 'weighted_ensemble_custom_pareto16_k0_l2', 'weighted_ensemble_custom_pareto17_k0_l2', 'weighted_ensemble_custom_pareto18_k0_l2', 'weighted_ensemble_custom_pareto19_k0_l2', 'weighted_ensemble_custom_pareto20_k0_l2', 'weighted_ensemble_custom_k0_l2']


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,weighted_ensemble_custom_k0_l2,0.915336,60.588694,9204.402478,0.063801,139.978314,2,True,45
1,weighted_ensemble_custom_pareto20_k0_l2,0.915336,60.473552,9195.161349,0.065026,134.269739,2,True,44
2,weighted_ensemble_custom_pareto19_k0_l2,0.915335,59.350921,9461.867956,0.062866,126.878810,2,True,43
3,weighted_ensemble_custom_pareto18_k0_l2,0.915313,54.540925,6816.983227,0.063772,120.053896,2,True,42
4,weighted_ensemble_custom_pareto17_k0_l2,0.915313,50.323075,6235.013777,0.064780,113.291757,2,True,41
5,weighted_ensemble_custom_pareto14_k0_l2,0.915284,46.237116,5456.276426,0.063834,93.461995,2,True,38
6,weighted_ensemble_custom_pareto13_k0_l2,0.915276,45.549471,5426.519485,0.064145,93.572446,2,True,37
7,weighted_ensemble_custom_pareto12_k0_l2,0.915237,45.159443,5404.759320,0.062866,87.062913,2,True,36
8,weighted_ensemble_custom_pareto11_k0_l2,0.915179,44.945945,5386.567959,0.065399,79.931511,2,True,35
9,CatboostClassifier_STACKER_l1,0.915090,44.880546,5306.636448,0.069749,38.222966,1,True,21


## Collapsing bagged ensembles via refit_full

In [19]:
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data, silent=True)

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\utils\data\X_train.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\utils\data\y_train.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_STACKER_l0\utils\model_template.pkl
Fitting model: RandomForestClassifierGini_FULL_STACKER_l0 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_FULL_STACKER_l0\utils\model_template.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_FULL_STACKER_l0\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierGini_FULL_STACKER_l0\model.pkl
	36.51s	 = Training runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\RandomForestClassifierEntr_STACKER_l0\model.pkl
Loadin

[50]	train_set's binary_logloss: 0.456131
[100]	train_set's binary_logloss: 0.433076
[150]	train_set's binary_logloss: 0.419268
[200]	train_set's binary_logloss: 0.409607
[250]	train_set's binary_logloss: 0.401856
[300]	train_set's binary_logloss: 0.395259
[350]	train_set's binary_logloss: 0.389263
[400]	train_set's binary_logloss: 0.383695
[450]	train_set's binary_logloss: 0.378566
[500]	train_set's binary_logloss: 0.373619


Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_FULL_STACKER_l0\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_FULL_STACKER_l0\model.pkl
	4.28s	 = Training runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_STACKER_l0\utils\model_template.pkl
Fitting model: LightGBMClassifierXT_FULL_STACKER_l0 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_FULL_STACKER_l0\utils\model_template.pkl
Training Gradient Boosting Model for 911 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'binary', 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'extra_trees': True}


[50]	train_set's binary_logloss: 0.467648
[100]	train_set's binary_logloss: 0.448989
[150]	train_set's binary_logloss: 0.438557
[200]	train_set's binary_logloss: 0.431028
[250]	train_set's binary_logloss: 0.425206
[300]	train_set's binary_logloss: 0.420399
[350]	train_set's binary_logloss: 0.416184
[400]	train_set's binary_logloss: 0.412485
[450]	train_set's binary_logloss: 0.409174
[500]	train_set's binary_logloss: 0.406185
[550]	train_set's binary_logloss: 0.403424
[600]	train_set's binary_logloss: 0.400716
[650]	train_set's binary_logloss: 0.398153
[700]	train_set's binary_logloss: 0.395703
[750]	train_set's binary_logloss: 0.39332
[800]	train_set's binary_logloss: 0.391082
[850]	train_set's binary_logloss: 0.388817
[900]	train_set's binary_logloss: 0.386674


Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_FULL_STACKER_l0\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_FULL_STACKER_l0\model.pkl
	7.1s	 = Training runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l0\utils\model_template.pkl
Fitting model: CatboostClassifier_FULL_STACKER_l0 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_FULL_STACKER_l0\utils\model_template.pkl
	Catboost model hyperparameters: {'iterations': 1129, 'learning_rate': 0.1, 'random_seed': 0, 'allow_writing_files': False, 'eval_metric': 'Logloss'}


...

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l0\utils\oof.pkl
Fitting model: weighted_ensemble_FULL_k0_l1 ...
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l1\utils\model_template.pkl
Ensemble size: 57
Ensemble weights: 
[0.96491228 0.03508772]
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l1\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l1\model.pkl
	0.915	 = Validation roc_auc score
	8.04s	 = Training runtime
	0.07s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l1\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l1\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto8_k0_l1\model.pkl
Loading: D:\Uni\202

Ensemble weights: 
[0.48837209 0.3255814  0.18604651]
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l2\utils\oof.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l2\model.pkl
	0.9152	 = Validation roc_auc score
	18.32s	 = Training runtime
	0.07s	 = Validation runtime
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\trainer.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l2\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l2\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto13_k0_l2\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto13_k0_l2\utils\model_template.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l2\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_FULL_k0_l2\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto18_k0_l2\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\weighted_ensemble_custom_pareto18_k0_l2\utils\model_template.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\CatboostClassifier_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\NeuralNetClassifier_STACKER_l0\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierXT_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifierCustom_STACKER_l1\utils\oof.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_mode

Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierGini_FULL_STACKER_l0\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierGini_FULL_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_FULL_STACKER_l0\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\ExtraTreesClassifierEntr_FULL_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_FULL_STACKER_l0\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierUnif_FULL_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierDist_FULL_STACKER_l0\model.pkl
Saving D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\KNeighborsClassifierDist_FULL_STACKER_l0\model.pkl
Loading: D:\Uni\2020.ml.kaggle\Marc\\trained_models\models\LightGBMClassifier_FULL_STACKER_l0\model.pkl
Saving D:\Uni\2020.ml.ka

KeyError: 'NeuralNetClassifier_FULL_STACKER_l0'

## Create csv

In [8]:
# We only get the train data set
current_path = os.getcwd()+"\\"
file = os.path.sep.join(["..\\data\\test.csv"])
data = pd.read_csv(current_path + file)
data=data.set_index("Id")

# signal is the target variable
df_test=data.drop(["BUTTER"], axis=1) # Droping BUTTER variable as is IRRELEVANT

In [9]:
y_pred = predictor.predict_proba(df_test)
y_pred

array([0.02818869, 0.03101594, 0.13268298, ..., 0.74875873, 0.7941227 ,
       0.03582523], dtype=float32)

In [10]:
results=pd.DataFrame(columns=["Predicted"],data=y_pred)
results.to_csv("WinResults4.csv")